Code adapted from NVIDIA OpenACC example: laplace2d-kernels.c

Copyright (c) 2012, NVIDIA CORPORATION. All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions
are met:
 * Redistributions of source code must retain the above copyright
   notice, this list of conditions and the following disclaimer.
 * Redistributions in binary form must reproduce the above copyright
   notice, this list of conditions and the following disclaimer in the
   documentation and/or other materials provided with the distribution.
 * Neither the name of NVIDIA CORPORATION nor the names of its
   contributors may be used to endorse or promote products derived
   from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS ''AS IS'' AND ANY
EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR
PURPOSE ARE DISCLAIMED.  IN NO EVENT SHALL THE COPYRIGHT OWNER OR
CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL,
EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO,
PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR
PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY
OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
(INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [1]:
%load_ext line_profiler

In [2]:
%load_ext snakeviz

In [3]:
import numpy as np
import time

In [4]:
import pickle

In [5]:
from numba import guvectorize

In [6]:
from math import fabs

In [7]:
from numba import jit, njit

In [8]:
pow_max = 13
N = [2 ** val for val in range(2, pow_max)]

In [5]:
def np_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = 1.0

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)

    while error > tol and iteration < iteration_max:
        error = 0.0
        Anew[1:-1, 1:-1] = 0.25 * (A[1:-1, 2:] + A[1:-1, :-2] + A[2:, 1:-1] + A[:-2, 1:-1])      
        diff = np.abs(Anew[1:-1, 1:-1] - A[1:-1, 1:-1])
        error_new = np.max(diff)
        if error_new > error:
            error = error_new
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error))

        iteration += 1

    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [6]:
res = np_laplace(12, 12)

In [7]:
%lprun -f np_laplace np_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.002395

  200, 0.001202

  300, 0.000801

  400, 0.000601

  500, 0.000481

  600, 0.000400

  700, 0.000343

  800, 0.000300

  900, 0.000266

 total: 2.3715338706970215 s



In [6]:
N

[4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]

In [10]:
np_res = {}
for n in N:
    res = %timeit -o -n 3 np_laplace(n, n)
    np_res[n] = res
pickle.dump(np_res, open('np_times.p', 'wb'))

3 loops, best of 3: 631 µs per loop
3 loops, best of 3: 2.74 ms per loop
3 loops, best of 3: 8.13 ms per loop
3 loops, best of 3: 27.2 ms per loop
3 loops, best of 3: 56.4 ms per loop
3 loops, best of 3: 154 ms per loop
3 loops, best of 3: 538 ms per loop
3 loops, best of 3: 2.02 s per loop
3 loops, best of 3: 10.1 s per loop
3 loops, best of 3: 56.7 s per loop
3 loops, best of 3: 4min 28s per loop


In [11]:
np_res

{4: <TimeitResult : 3 loops, best of 3: 631 µs per loop>,
 8: <TimeitResult : 3 loops, best of 3: 2.74 ms per loop>,
 16: <TimeitResult : 3 loops, best of 3: 8.13 ms per loop>,
 32: <TimeitResult : 3 loops, best of 3: 27.2 ms per loop>,
 64: <TimeitResult : 3 loops, best of 3: 56.4 ms per loop>,
 128: <TimeitResult : 3 loops, best of 3: 154 ms per loop>,
 256: <TimeitResult : 3 loops, best of 3: 538 ms per loop>,
 512: <TimeitResult : 3 loops, best of 3: 2.02 s per loop>,
 1024: <TimeitResult : 3 loops, best of 3: 10.1 s per loop>,
 2048: <TimeitResult : 3 loops, best of 3: 56.7 s per loop>,
 4096: <TimeitResult : 3 loops, best of 3: 4min 28s per loop>}

# Adding numba jit decorator

In [13]:
@jit(['float32[:,:], float32[:,:]',
      'float64[:,:], float64[:,:]'],
      nopython=True)
def update_a_jit(A, Anew):
    error = 0.0
    n, m = A.shape

    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])
            error = max(error, abs(Anew[j, i] - A[j, i]))

    return Anew, error

In [14]:
# this did not make much difference
@jit(['float32[:,:], float32[:,:]',
      'float64[:,:], float64[:,:]'],
      nopython=True)
def a_to_anew_jit(A, Anew):
    error = 0.0
    n, m = A.shape
    
    for j in range(1, n-1):
        for i in range(1, m-1):
            A[j, i] = Anew[j, i]
    
    return A

In [15]:
def jit_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = 1.0

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)

    while error > tol and iteration < iteration_max:
        Anew, error = update_a_jit(A, Anew)
        # A[:] = Anew[:]
        A = a_to_anew_jit(A, Anew)  # this was essentially the same as the line above

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error))

        iteration += 1

    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [16]:
res = jit_laplace(512, 512) # once to compile

In [17]:
%lprun -f jit_laplace jit_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.002395

  200, 0.001202

  300, 0.000801

  400, 0.000601

  500, 0.000481

  600, 0.000400

  700, 0.000343

  800, 0.000300

  900, 0.000266

 total: 0.7033450603485107 s



In [18]:
jit_res = {}
for n in N:
    res = %timeit -o -n 3 jit_laplace(n, n)
    jit_res[n] = res
pickle.dump(jit_res, open('jit_times.p', 'wb'))

3 loops, best of 3: 47.4 µs per loop
3 loops, best of 3: 128 µs per loop
3 loops, best of 3: 535 µs per loop
3 loops, best of 3: 3.38 ms per loop
3 loops, best of 3: 9.89 ms per loop
3 loops, best of 3: 35.7 ms per loop
3 loops, best of 3: 159 ms per loop
3 loops, best of 3: 669 ms per loop
3 loops, best of 3: 2.7 s per loop
3 loops, best of 3: 14.1 s per loop
3 loops, best of 3: 55.9 s per loop


# Implementing `guvectorize`

### CPU Vectorization

In [21]:
@guvectorize(['float32[:,:], float32[:,:]',
              'float64[:,:], float64[:,:]'],
             '(n,m)->(n,m)', nopython=True,
             target='cpu')
def update_a_serial0(A, Anew):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

In [22]:
@guvectorize(['float32[:,:], float32[:,:], float32[:]',
              'float64[:,:], float64[:,:], float64[:]'],
             '(n,m)->(n,m),()', nopython=True, target='cpu')
def update_a_serial(A, Anew, error):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

            diff = abs(Anew[j, i] - A[j, i])
            if diff > error[0]:
                error[0] = diff

In [23]:
A = np.zeros((4,4))
A[:, [0, -1]] = 1.0
Anew = np.zeros_like(A)
error = np.zeros(1)
Anew, error = update_a_serial(A, Anew, error)
np.max(np.abs(Anew[1:-1, 1:-1] - A[1:-1, 1:-1])), error[0]

(0.25, 0.25)

In [24]:
def cpu_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = np.ones(1)

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)
    while error[0] > tol and iteration < iteration_max:
        error[0] = 0.0
        Anew, error = update_a_serial(A, Anew, error)
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error[0]))

        iteration += 1

    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [25]:
res = cpu_laplace(512, 512)

In [26]:
%lprun -f cpu_laplace cpu_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.002395

  200, 0.001202

  300, 0.000801

  400, 0.000601

  500, 0.000481

  600, 0.000400

  700, 0.000343

  800, 0.000300

  900, 0.000266

 total: 2.6767690181732178 s



In [27]:
%timeit cpu_laplace(1024, 1024)

1 loop, best of 3: 8 s per loop


In [28]:
cpu_res = {}
for n in N:
    res = %timeit -o -n 3 cpu_laplace(n, n)
    cpu_res[n] = res
pickle.dump(cpu_res, open('cpu_times.p', 'wb'))

3 loops, best of 3: 121 µs per loop
3 loops, best of 3: 502 µs per loop
3 loops, best of 3: 897 µs per loop
3 loops, best of 3: 4.22 ms per loop
3 loops, best of 3: 10.9 ms per loop
3 loops, best of 3: 42.2 ms per loop
3 loops, best of 3: 169 ms per loop
3 loops, best of 3: 2.57 s per loop
3 loops, best of 3: 8.63 s per loop
3 loops, best of 3: 13.6 s per loop
3 loops, best of 3: 53.2 s per loop


### parallel CPU vectorization

In [29]:
@guvectorize(['float32[:,:], float32[:,:], float32[:]',
              'float64[:,:], float64[:,:], float64[:]'],
             '(n,m)->(n,m),()', nopython=True, target='parallel')
def update_a_parallel(A, Anew, error):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

            diff = abs(Anew[j, i] - A[j, i])
            if diff > error[0]:
                error[0] = diff

In [30]:
def par_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    pi = np.pi
    tol = 1.0e-5
    error = np.ones(1)

    A = np.zeros((n, m))

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)
    while error[0] > tol and iteration < iteration_max:
        error[0] = 0.0
        Anew, error = update_a_parallel(A, Anew, error)
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error[0]))

        iteration += 1

    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [31]:
res = par_laplace(512, 512, debug=False)

In [32]:
%lprun -f par_laplace cpu_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.002395

  200, 0.001202

  300, 0.000801

  400, 0.000601

  500, 0.000481

  600, 0.000400

  700, 0.000343

  800, 0.000300

  900, 0.000266

 total: 2.7831718921661377 s



In [33]:
%timeit par_laplace(1024, 1024)

1 loop, best of 3: 7.29 s per loop


In [34]:
par_res = {}
for n in N:
    res = %timeit -o -n 3 par_laplace(n, n)
    par_res[n] = res
pickle.dump(par_res, open('par_times.p', 'wb'))

3 loops, best of 3: 1.93 ms per loop
3 loops, best of 3: 8.3 ms per loop
3 loops, best of 3: 26.3 ms per loop
3 loops, best of 3: 73.8 ms per loop
3 loops, best of 3: 70.8 ms per loop
3 loops, best of 3: 108 ms per loop
3 loops, best of 3: 268 ms per loop
3 loops, best of 3: 3.5 s per loop
3 loops, best of 3: 10.1 s per loop
3 loops, best of 3: 15.7 s per loop
3 loops, best of 3: 1min 27s per loop


In [40]:
np_res, jit_res, cpu_res, par_res

({4: <TimeitResult : 3 loops, best of 3: 631 µs per loop>,
  8: <TimeitResult : 3 loops, best of 3: 2.74 ms per loop>,
  16: <TimeitResult : 3 loops, best of 3: 8.13 ms per loop>,
  32: <TimeitResult : 3 loops, best of 3: 27.2 ms per loop>,
  64: <TimeitResult : 3 loops, best of 3: 56.4 ms per loop>,
  128: <TimeitResult : 3 loops, best of 3: 154 ms per loop>,
  256: <TimeitResult : 3 loops, best of 3: 538 ms per loop>,
  512: <TimeitResult : 3 loops, best of 3: 2.02 s per loop>,
  1024: <TimeitResult : 3 loops, best of 3: 10.1 s per loop>,
  2048: <TimeitResult : 3 loops, best of 3: 56.7 s per loop>,
  4096: <TimeitResult : 3 loops, best of 3: 4min 28s per loop>},
 {4: <TimeitResult : 3 loops, best of 3: 47.4 µs per loop>,
  8: <TimeitResult : 3 loops, best of 3: 128 µs per loop>,
  16: <TimeitResult : 3 loops, best of 3: 535 µs per loop>,
  32: <TimeitResult : 3 loops, best of 3: 3.38 ms per loop>,
  64: <TimeitResult : 3 loops, best of 3: 9.89 ms per loop>,
  128: <TimeitResult : 3 

In [42]:
with open('par_times.p', 'rb') as f:
    par_load = pickle.load(f)

In [44]:
par_load, par_res

({4: <TimeitResult : 3 loops, best of 3: 1.93 ms per loop>,
  8: <TimeitResult : 3 loops, best of 3: 8.3 ms per loop>,
  16: <TimeitResult : 3 loops, best of 3: 26.3 ms per loop>,
  32: <TimeitResult : 3 loops, best of 3: 73.8 ms per loop>,
  64: <TimeitResult : 3 loops, best of 3: 70.8 ms per loop>,
  128: <TimeitResult : 3 loops, best of 3: 108 ms per loop>,
  256: <TimeitResult : 3 loops, best of 3: 268 ms per loop>,
  512: <TimeitResult : 3 loops, best of 3: 3.5 s per loop>,
  1024: <TimeitResult : 3 loops, best of 3: 10.1 s per loop>,
  2048: <TimeitResult : 3 loops, best of 3: 15.7 s per loop>,
  4096: <TimeitResult : 3 loops, best of 3: 1min 27s per loop>},
 {4: <TimeitResult : 3 loops, best of 3: 1.93 ms per loop>,
  8: <TimeitResult : 3 loops, best of 3: 8.3 ms per loop>,
  16: <TimeitResult : 3 loops, best of 3: 26.3 ms per loop>,
  32: <TimeitResult : 3 loops, best of 3: 73.8 ms per loop>,
  64: <TimeitResult : 3 loops, best of 3: 70.8 ms per loop>,
  128: <TimeitResult : 3 

### CUDA vectorization

In [35]:
@guvectorize(['float32[:,:], float32[:,:]',
              'float64[:,:], float64[:,:]'],
             '(n,m)->(n,m)', nopython=True, target='cuda')
def update_a_cuda(A, Anew):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])

In [36]:
@guvectorize(['float32[:,:], float32[:,:], float32[:]',
              'float64[:,:], float64[:,:], float64[:]'],
             '(n,m),(n,m)->()', nopython=True, target='cuda')
def update_error_cuda(A, Anew, error):
    n, m = A.shape
    for j in range(1, n-1):
        for i in range(1, m-1):
            diff = abs(Anew[j, i] - A[j, i])
            if diff > error[0]:
                error[0] = diff

In [38]:
a = np.arange(16, dtype=np.float32).reshape(4, 4)
anew = np.zeros((4, 4), dtype=np.float32)
anew[1:-1, 1:-1] = update_a_cuda(a)[1:-1, 1:-1]
anew

array([[  0.,   0.,   0.,   0.],
       [  0.,   5.,   6.,   0.],
       [  0.,   9.,  10.,   0.],
       [  0.,   0.,   0.,   0.]], dtype=float32)

In [39]:
def cuda_laplace(n=4096, m=4096, iteration_max = 1000, debug=False):

    dtype=np.float32
    
    pi = np.pi
    tol = 1.0e-5
    error = np.ones(1)
    
    A = np.zeros((n, m), dtype=dtype)

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iteration = 0

    Anew = np.copy(A)
    while error[0] > tol and iteration < iteration_max:
        error[0] = 0.0
        res = update_a_cuda(A)
        Anew[1:-1, 1:-1] = res[1:-1, 1:-1]
        error = update_error_cuda(A, Anew)
        A[:] = Anew[:]

        if debug and not iteration % 100:
            print("{:5d}, {:0.6f}\n".format(iteration, error[0]))

        iteration += 1

    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [40]:
res = cuda_laplace(512, 512, debug=False)

In [41]:
%lprun -f cuda_laplace cuda_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

    0, 0.249999

  100, 0.124997

  200, 0.249999

  300, 0.054685

  400, 0.124997

  500, 0.249999

  600, 0.124997

  700, 0.054685

  800, 0.000300

  900, 0.000266

 total: 145.69989371299744 s



In [44]:
cuda_res = {}
for n in N[:6]:
    res = %timeit -o -n 3 cuda_laplace(n, n)
    cuda_res[n] = res
pickle.dump(cuda_res, open('cuda_times.p', 'wb'))

3 loops, best of 3: 32.7 ms per loop
3 loops, best of 3: 132 ms per loop
3 loops, best of 3: 525 ms per loop
3 loops, best of 3: 2.23 s per loop
3 loops, best of 3: 3.75 s per loop
3 loops, best of 3: 10.5 s per loop


This is much slower, likely because of data transfer after each iteration.  I will try to put the entire while loop on the GPU.

In [9]:
@guvectorize([
        'float32[:,:], float32[:,:]',
        'float64[:,:], float64[:,:]',
    ], '(n,m)->(n,m)', target='cuda', nopython=True)
def iterate_a(A, Anew):
    n, m = A.shape
    tol = 1.0e-5
    iteration_max = 1000
    error = 1.0
    iteration = 0

    # main loop
    while error > tol and iteration < iteration_max:
        error = 0.0
        for j in range(1, n-1):
            for i in range(1, m-1):
                Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])
                diff = abs(Anew[j, i] - A[j, i])
                if diff > error:
                    error = diff
        
        for j in range(1, n-1):
            for i in range(1, m-1):
                A[j, i] = Anew[j, i]

        iteration += 1

In [10]:
def cuda_laplace(n=4096, m=4096, debug=False):

    dtype=np.float32
    
    pi = np.pi
    
    A = np.zeros((n, m), dtype=dtype)

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    iterate_a(A)
    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [14]:
res = cuda_laplace(16, 16, debug=True)

Jacobi relaxation Calculation: 16 x 16 mesh

 total: 0.06943607330322266 s



In [15]:
res

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  2.07911685e-01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          8.98467842e-03],
       [  4.06736642e-01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.7576

In [17]:
# think this crashes...
# %lprun -f cuda_laplace cuda_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

In [26]:
N[:3]

[4, 8, 16]

In [19]:
cuda_res = {}
for n in N:
    res = %timeit -o -n 3 cuda_laplace(n, n)
    cuda_res[n] = res
pickle.dump(cuda_res, open('cuda_times.p', 'wb'))

3 loops, best of 3: 1.59 ms per loop
3 loops, best of 3: 4.28 ms per loop
3 loops, best of 3: 57.3 ms per loop
3 loops, best of 3: 846 ms per loop
3 loops, best of 3: 3.58 s per loop


CudaAPIError: [702] Call to cuMemcpyDtoH results in CUDA_ERROR_LAUNCH_TIMEOUT

## use this with CPU and Parallel

In [20]:
@guvectorize([
        'float32[:,:], float32[:,:]',
        'float64[:,:], float64[:,:]',
    ], '(n,m)->(n,m)', target='cpu', nopython=True)
def cpu_iterate_a(A, Anew):
    n, m = A.shape
    tol = 1.0e-5
    iteration_max = 1000
    error = 1.0
    iteration = 0

    # main loop
    while error > tol and iteration < iteration_max:
        error = 0.0
        for j in range(1, n-1):
            for i in range(1, m-1):
                Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])
                diff = abs(Anew[j, i] - A[j, i])
                if diff > error:
                    error = diff
        
        for j in range(1, n-1):
            for i in range(1, m-1):
                A[j, i] = Anew[j, i]

        iteration += 1

In [21]:
def cpu_laplace(n=4096, m=4096, debug=False):

    dtype=np.float32
    
    pi = np.pi
    
    A = np.zeros((n, m), dtype=dtype)

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    cpu_iterate_a(A)
    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [22]:
res = cpu_laplace(16, 16, debug=True)

Jacobi relaxation Calculation: 16 x 16 mesh

 total: 0.00041365623474121094 s



In [24]:
res[:4, :4]

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.20791169,  0.16873163,  0.13692765,  0.11111137],
       [ 0.40673664,  0.33008888,  0.2678709 ,  0.21736664],
       [ 0.58778524,  0.47701967,  0.3871069 ,  0.31412193]], dtype=float32)

In [26]:
# think this crashes...
%lprun -f cpu_laplace cpu_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

Jacobi relaxation Calculation: 512 x 512 mesh

 total: 1.9225044250488281 s



In [ ]:
cpu_res = {}
for n in N:
    res = %timeit -o -n 3 cpu_laplace(n, n)
    cpu_res[n] = res
pickle.dump(cpu_res, open('cpu_times_v2.p', 'wb'))

3 loops, best of 3: 49.3 µs per loop
3 loops, best of 3: 68 µs per loop
3 loops, best of 3: 436 µs per loop
3 loops, best of 3: 4.35 ms per loop
3 loops, best of 3: 12.8 ms per loop
3 loops, best of 3: 50.3 ms per loop
3 loops, best of 3: 197 ms per loop
3 loops, best of 3: 1.68 s per loop
3 loops, best of 3: 6.54 s per loop


In [ ]:
@guvectorize([
        'float32[:,:], float32[:,:]',
        'float64[:,:], float64[:,:]',
    ], '(n,m)->(n,m)', target='parallel', nopython=True)
def par_iterate_a(A, Anew):
    n, m = A.shape
    tol = 1.0e-5
    iteration_max = 1000
    error = 1.0
    iteration = 0

    # main loop
    while error > tol and iteration < iteration_max:
        error = 0.0
        for j in range(1, n-1):
            for i in range(1, m-1):
                Anew[j, i] = 0.25 * (A[j, i+1] + A[j, i-1] + A[j-1, i] + A[j+1, i])
                diff = abs(Anew[j, i] - A[j, i])
                if diff > error:
                    error = diff
        
        for j in range(1, n-1):
            for i in range(1, m-1):
                A[j, i] = Anew[j, i]

        iteration += 1

In [ ]:
def par_laplace(n=4096, m=4096, debug=False):

    dtype=np.float32
    
    pi = np.pi
    
    A = np.zeros((n, m), dtype=dtype)

    j_range = np.arange(n)
    y0 = np.sin(pi * j_range / (n - 1))
    A[:, 0]  = y0
    A[:, -1] = y0 * np.exp(-pi)

    if debug: print('Jacobi relaxation Calculation: {} x {} mesh\n'.format(n, m))

    tic = time.time()
    par_iterate_a(A)
    toc = time.time() - tic

    if debug: print(" total: {} s\n".format(toc))
    
    return A

In [ ]:
res = par_laplace(16, 16, debug=True)

In [ ]:
res[:4, :4]

In [ ]:
# think this crashes...
%lprun -f par_laplace par_laplace(512,512,debug=True) # do not use spaces, it confuses the profiler

In [ ]:
par_res = {}
for n in N:
    res = %timeit -o -n 3 par_laplace(n, n)
    par_res[n] = res
pickle.dump(par_res, open('par_times_v2.p', 'wb'))